<a href="https://colab.research.google.com/github/kshero18/Assignments-ML/blob/main/Methodology_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00


In [2]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.1 MB/s eta 0:00:00


In [3]:
!pip install pydicom matplotlib pandas tensorflow scikit-learn

# Install necessary libraries
!pip install pydicom SimpleITK matplotlib kaggle

# Set up Kaggle authentication
from google.colab import files
files.upload()  # Upload kaggle.json file

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the RSNA dataset
!kaggle competitions download -c rsna-2024-lumbar-spine-degenerative-classification
!unzip -q rsna-2024-lumbar-spine-degenerative-classification.zip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 8.5 MB/s eta 0:00:00


Saving kaggle.json to kaggle (1).json
100% 28.2G/28.2G [04:51<00:00, 157MB/s]
100% 28.2G/28.2G [04:51<00:00, 104MB/s]


In [2]:
import os
import pandas as pd

# Load the CSV files
train_label_coordinates_df = pd.read_csv('/content/train_label_coordinates.csv')
train_data_df = pd.read_csv('/content/train.csv')
train_series_descriptions_df = pd.read_csv('/content/train_series_descriptions.csv')

# Merge training data (based on common `study_id` and `series_id`)
merged_train_df = pd.merge(train_label_coordinates_df, train_data_df, on='study_id')
merged_train_df = pd.merge(merged_train_df, train_series_descriptions_df, on=['study_id', 'series_id'])

# Verify the merged data
print(merged_train_df.head())

test_series_descriptions_df = pd.read_csv('/content/test_series_descriptions.csv')

   study_id  series_id  instance_number              condition  level  \
0   4003253  702807833                8  Spinal Canal Stenosis  L1/L2   
1   4003253  702807833                8  Spinal Canal Stenosis  L2/L3   
2   4003253  702807833                8  Spinal Canal Stenosis  L3/L4   
3   4003253  702807833                8  Spinal Canal Stenosis  L4/L5   
4   4003253  702807833                8  Spinal Canal Stenosis  L5/S1   

            x           y spinal_canal_stenosis_l1_l2  \
0  322.831858  227.964602                 Normal/Mild   
1  320.571429  295.714286                 Normal/Mild   
2  323.030303  371.818182                 Normal/Mild   
3  335.292035  427.327434                 Normal/Mild   
4  353.415929  483.964602                 Normal/Mild   

  spinal_canal_stenosis_l2_l3 spinal_canal_stenosis_l3_l4  ...  \
0                 Normal/Mild                 Normal/Mild  ...   
1                 Normal/Mild                 Normal/Mild  ...   
2                 No

In [3]:
import os
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Function to convert DICOM to JPEG and resize the image
def dicom_to_jpeg(dicom_file, output_path, target_size=(64, 64)):
    try:
        dicom = pydicom.dcmread(dicom_file)
        img = dicom.pixel_array

        # Normalize pixel values if necessary
        if img.dtype != np.uint8:
            img = img.astype(np.uint8)

        # Convert to PIL image
        img_pil = Image.fromarray(img)

        # Resize the image
        img_resized = img_pil.resize(target_size)

        # Save as JPEG
        img_resized.save(output_path)
    except Exception as e:
        print(f"Error processing {dicom_file}: {e}")

# Function to find DICOM files in subdirectories based on series_id in the directory path
def find_dicom_file(base_dir, series_id):
    for root, dirs, files in os.walk(base_dir):
        if str(series_id) in root:
            for file in files:
                if file.endswith('.dcm'):
                    return os.path.join(root, file)
    return None

# Function to normalize coordinates based on resized image size
def normalize_coordinates(x, y, original_size, target_size):
    try:
        x_norm = x * (target_size[0] / original_size[0])
        y_norm = y * (target_size[1] / original_size[1])
        return x_norm, y_norm
    except Exception as e:
        print(f"Error normalizing coordinates: {e}")
        return None, None

# Function to load JPEG images and normalize coordinates
def load_images_and_normalize_coords(df, base_dicom_path, jpeg_output_dir, target_size=(64, 64)):
    images = []
    coordinates = []

    for idx, row in df.iterrows():
        # Access 'x' and 'y' columns dynamically using .get() to avoid KeyError
        x_val = row.get('x')
        y_val = row.get('y')

        if pd.isna(x_val) or pd.isna(y_val):
            print(f"Skipping row {idx} due to missing coordinates: x={x_val}, y={y_val}")
            continue

        print(f"Processing row {idx}: x={x_val}, y={y_val}")

        # Search for the DICOM file in subdirectories
        dicom_file = find_dicom_file(base_dicom_path, row['series_id'])

        if dicom_file is None:
            print(f"DICOM file for series ID {row['series_id']} not found, skipping.")
            continue

        jpeg_file = os.path.join(jpeg_output_dir, f"{row['series_id']}.jpg")

        # Convert DICOM to JPEG
        dicom_to_jpeg(dicom_file, jpeg_file, target_size=target_size)

        # Load the resized JPEG image
        if os.path.exists(jpeg_file):
            img = Image.open(jpeg_file)
            img_array = np.array(img)
            images.append(img_array)

            # Normalize the coordinates based on resizing
            original_size = pydicom.dcmread(dicom_file).pixel_array.shape
            x_norm, y_norm = normalize_coordinates(x_val, y_val, original_size, target_size)
            coordinates.append([x_norm, y_norm])
        else:
            print(f"JPEG file {jpeg_file} not created.")

    return np.array(images), np.array(coordinates)

# Load a small sample (e.g., first 10 rows) of training images and normalized coordinates
X_train_images, train_coords = load_images_and_normalize_coords(merged_train_df.head(10), '/content/train_images/', '/content/jpeg_images/')

# Preprocess labels (assume 'condition' is the label for multi-class classification)
le = LabelEncoder()
y_train = le.fit_transform(merged_train_df.head(10)['condition'])
y_train = to_categorical(y_train)

# Split data into training and validation sets
if len(X_train_images) == len(train_coords) == len(y_train):
    X_train, X_val, train_coords_train, train_coords_val, y_train, y_val = train_test_split(
        X_train_images, train_coords, y_train, test_size=0.2, random_state=42
    )
else:
    print(f"Data mismatch: Images({len(X_train_images)}), Coordinates({len(train_coords)}), Labels({len(y_train)})")
    # Exit or handle mismatch error
    exit()

# Define CNN model with coordinates as additional input
def create_cnn_model(image_input_shape, coord_input_shape):
    # Image input branch
    image_input = Input(shape=image_input_shape, name='image_input')
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Coordinate input branch
    coord_input = Input(shape=coord_input_shape, name='coords_input')

    # Combine both inputs
    combined = Concatenate()([x, coord_input])

    # Fully connected layers
    z = Dense(128, activation='relu')(combined)
    z = Dropout(0.5)(z)
    z = Dense(y_train.shape[1], activation='softmax')(z)

    model = Model(inputs=[image_input, coord_input], outputs=z)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Reshape the training data to match CNN input requirements
X_train_images_resized = X_train.reshape(-1, 64, 64, 1)
X_val_images_resized = X_val.reshape(-1, 64, 64, 1)

# Build and train the model
input_shape = (64, 64, 1)
coord_shape = (2,)
model = create_cnn_model(input_shape, coord_shape)

# Train the model
model.fit([X_train_images_resized, train_coords_train], y_train, epochs=10, batch_size=32, validation_data=([X_val_images_resized, train_coords_val], y_val))

Processing row 0: x=322.83185840707966, y=227.9646017699115
Error processing /content/train_images/4003253/702807833/9.dcm: [Errno 2] No such file or directory: '/content/jpeg_images/702807833.jpg'
JPEG file /content/jpeg_images/702807833.jpg not created.
Processing row 1: x=320.57142857142856, y=295.7142857142857
Error processing /content/train_images/4003253/702807833/9.dcm: [Errno 2] No such file or directory: '/content/jpeg_images/702807833.jpg'
JPEG file /content/jpeg_images/702807833.jpg not created.
Processing row 2: x=323.03030303030306, y=371.8181818181818
Error processing /content/train_images/4003253/702807833/9.dcm: [Errno 2] No such file or directory: '/content/jpeg_images/702807833.jpg'
JPEG file /content/jpeg_images/702807833.jpg not created.
Processing row 3: x=335.2920353982301, y=427.3274336283186
Error processing /content/train_images/4003253/702807833/9.dcm: [Errno 2] No such file or directory: '/content/jpeg_images/702807833.jpg'
JPEG file /content/jpeg_images/7028

NameError: name 'X_train' is not defined

In [ ]:
# Print the column names of the dataframe to verify
print(merged_train_df.columns)


In [ ]:
# Load the test images and normalized coordinates from test_series_descriptions_df
X_test_images, test_coords = load_images_and_normalize_coords(test_series_descriptions_df, '/content/test_images/', '/content/jpeg_images/')

# Reshape test images for prediction
X_test_images_resized = X_test_images.reshape(-1, 64, 64, 1)

# Create dummy coordinates for the test set since they are missing
dummy_coords = np.zeros((X_test_images_resized.shape[0], 2))  # Assuming 2D coordinates (X, Y)

# Check if test data is available before making predictions
print(f"Test images shape: {X_test_images_resized.shape}")
print(f"Test coordinates shape: {dummy_coords.shape}")

if X_test_images_resized.size == 0 or dummy_coords.size == 0:
    print("Test data is empty. Cannot make predictions.")
else:
    # Make predictions using the test images and dummy coordinates
    y_pred = model.predict([X_test_images_resized, dummy_coords])

    # Function to calculate F1 score
    def evaluate_model_with_f1(y_true, y_pred):
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_true, axis=1)
        f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
        print(f"F1 Score (weighted): {f1}")

    # Assuming you have the true labels for the test set
    # y_test_encoded = ...  # Load or prepare the true labels for the test set
    # evaluate_model_with_f1(y_test_encoded, y_pred)


TRYING TO FIND THE X AND Y COORDINATES IN TEST DATA USING THE METADATA

In [ ]:
import pydicom

def inspect_dicom_metadata(dicom_file):
    dicom_data = pydicom.dcmread(dicom_file)
    print(dicom_data)  # Print all metadata

# Test image
inspect_dicom_metadata('/content/test_images/44036939/2828203845/1.dcm')

# Train image
# inspect_dicom_metadata('/content/train_images/100206310/1012284084/1.dcm')


In [ ]:
import numpy as np

def calculate_patient_position(image_position, image_orientation, pixel_spacing, row_idx, col_idx):
    # Extract row and column vectors from image_orientation
    row_vector = np.array([float(image_orientation[0]), float(image_orientation[1]), float(image_orientation[2])])
    col_vector = np.array([float(image_orientation[3]), float(image_orientation[4]), float(image_orientation[5])])

    # Convert image position (patient) to a numpy array
    image_position = np.array([float(image_position[0]), float(image_position[1]), float(image_position[2])])

    # Pixel spacing for rows and columns
    pixel_spacing_row = float(pixel_spacing[0])
    pixel_spacing_col = float(pixel_spacing[1])

    # Calculate the position in 3D space for the pixel at (row_idx, col_idx)
    patient_position = image_position + (row_idx * pixel_spacing_row * row_vector) + (col_idx * pixel_spacing_col * col_vector)

    return patient_position

# DICOM metadata from the test file
image_position_patient = [50.3217, -81.2502, 156.63]  # Image Position (Patient)
image_orientation_patient = [2.22e-13, 1, 4.892e-12, -0.045295, 4.897e-12, -0.998974]  # Image Orientation (Patient)
pixel_spacing = [0.375, 0.375]  # Pixel Spacing

# Example pixel coordinates (center of the image or any region of interest)
row_idx = 320  # Row index (in the middle of the image)
col_idx = 320  # Column index (in the middle of the image)

# Calculate the physical position in the patient's coordinate system for the center of the area
patient_position = calculate_patient_position(image_position_patient, image_orientation_patient, pixel_spacing, row_idx, col_idx)

print(f"Patient position for pixel ({row_idx}, {col_idx}): {patient_position}")


In [ ]:
import pydicom
import cv2
import numpy as np

# Function to find the center of a region in the DICOM image
def find_center_of_region(image):
    # Convert image to grayscale if necessary (since DICOM images are often grayscale already)
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it's already a single channel (grayscale), use it directly

    # Use edge detection to find the region's boundaries
    edges = cv2.Canny(gray_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the region of interest
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the center of the contour
        moments = cv2.moments(largest_contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])  # x coordinate of the center
            cy = int(moments['m01'] / moments['m00'])  # y coordinate of the center
            return cx, cy
        else:
            return None, None
    else:
        return None, None

# Read the DICOM file using pydicom
dicom_file = '/content/train_images/100206310/1012284084/1.dcm'
dicom_data = pydicom.dcmread(dicom_file)

# Get the pixel data from the DICOM file
image_array = dicom_data.pixel_array

# Normalize the pixel data (optional, depends on your image)
image_array = cv2.normalize(image_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# Find the center of the region in the DICOM image
center_x, center_y = find_center_of_region(image_array)

if center_x is not None and center_y is not None:
    print(f"Center of region: x={center_x}, y={center_y}")
else:
    print("No region found or invalid DICOM data.")


TRYING TO FIND THE CENTRE OF THE REGION

In [ ]:
import os
import pydicom
import cv2
import numpy as np
import pandas as pd

# Function to find the center of a region in the DICOM image
def find_center_of_region(image):
    # Convert image to grayscale if necessary (since DICOM images are often grayscale already)
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # If it's already grayscale, use it directly

    # Use edge detection to find the region's boundaries
    edges = cv2.Canny(gray_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the region of interest
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the center of the contour
        moments = cv2.moments(largest_contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])  # x coordinate of the center
            cy = int(moments['m01'] / moments['m00'])  # y coordinate of the center
            return cx, cy
        else:
            return None, None
    else:
        return None, None

# Path to your test DICOM images
test_images_path = '/content/test_images/'

# List to store the test image data and coordinates
test_image_data = []
test_coords_data = []

# Iterate over each test image
for root, dirs, files in os.walk(test_images_path):
    for file in files:
        if file.endswith('.dcm'):
            dicom_file = os.path.join(root, file)

            # Read the DICOM file using pydicom
            dicom_data = pydicom.dcmread(dicom_file)

            # Get the pixel data from the DICOM file
            image_array = dicom_data.pixel_array

            # Normalize the pixel data (optional, depends on your image)
            image_array = cv2.normalize(image_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

            # Resize the image to match the model input (e.g., 64x64)
            resized_image = cv2.resize(image_array, (64, 64))

            # Find the center of the region in the DICOM image
            center_x, center_y = find_center_of_region(image_array)

            # Append the resized image and coordinates to the lists
            if center_x is not None and center_y is not None:
                test_image_data.append(resized_image)
                test_coords_data.append([center_x, center_y])
            else:
                print(f"Skipping {dicom_file} due to missing coordinates.")

# Convert to NumPy arrays
X_test_images_resized = np.array(test_image_data).reshape(-1, 64, 64, 1)  # Reshape for the CNN input
test_coords = np.array(test_coords_data)  # Coordinates for each image

# Ensure that both images and coordinates are non-empty
if X_test_images_resized.size == 0 or test_coords.size == 0:
    print("No valid test data available for prediction.")
else:
    # Make predictions using the test images and their coordinates
    y_pred = model.predict([X_test_images_resized, test_coords])

    # # Function to calculate F1 score (assuming you have true labels)
    # def evaluate_model_with_f1(y_true, y_pred):
    #     y_pred_classes = np.argmax(y_pred, axis=1)
    #     y_true_classes = np.argmax(y_true, axis=1)
    #     f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
    #     print(f"F1 Score (weighted): {f1}")

    # # Assuming you have the true labels for the test set (e.g., y_test_encoded)
    # evaluate_model_with_f1(y_test_encoded, y_pred)

    # Save the predictions if needed
    print("Predictions made successfully!")

y_pred

In [ ]:
import pandas as pd
import os
import pydicom
import cv2

def get_condition_from_series_description(series_description):
    # Example logic to assign condition based on series description
    series_description = series_description.lower()  # Convert to lowercase for consistent matching

    if "t1" in series_description:
        return "T1"
    elif "t2" in series_description:
        return "T2"
    elif "sagittal" in series_description:
        return "Sagittal"
    elif "axial" in series_description:
        return "Axial"
    else:
        return "Unknown_Condition"

# Function to find the center of a region in the DICOM image
def find_center_of_region(image):
    # Convert image to grayscale if necessary
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # Already grayscale

    # Use edge detection to find the region's boundaries
    edges = cv2.Canny(gray_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the region of interest
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the center of the contour
        moments = cv2.moments(largest_contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])  # x coordinate of the center
            cy = int(moments['m01'] / moments['m00'])  # y coordinate of the center
            return cx, cy
    return None, None

# Load the test_series_descriptions.csv
test_series_descriptions_df = pd.read_csv('/content/test_series_descriptions.csv')

# List to store the rows for the final CSV
test_data_rows = []

# Path to your test DICOM images
test_images_path = '/content/test_images/'

# Iterate over each test image and calculate coordinates
for root, dirs, files in os.walk(test_images_path):
    for file in files:
        if file.endswith('.dcm'):
            dicom_file = os.path.join(root, file)

            # Read the DICOM file using pydicom
            dicom_data = pydicom.dcmread(dicom_file)

            # Get the pixel data from the DICOM file
            image_array = dicom_data.pixel_array

            # Normalize the pixel data (optional)
            image_array = cv2.normalize(image_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

            # Find the center of the region in the DICOM image
            center_x, center_y = find_center_of_region(image_array)

            if center_x is not None and center_y is not None:
                # Extract metadata from DICOM file for the CSV
                study_id = dicom_data.PatientID
                series_id = dicom_data.SeriesInstanceUID.split('.')[-1]  # Extract just the series number
                instance_number = dicom_data.InstanceNumber

                # Find corresponding series description from the test_series_descriptions.csv
                series_info = test_series_descriptions_df[
                    (test_series_descriptions_df['study_id'] == str(study_id)) &
                    (test_series_descriptions_df['series_id'] == str(series_id))
                ]

                # Get the series description and determine the condition
                if not series_info.empty:
                    series_description = series_info.iloc[0]['series_description']
                    condition = get_condition_from_series_description(series_description)
                else:
                    condition = "Unknown_Condition"

                # Append the data to the list
                test_data_rows.append({
                    'study_id': study_id,
                    'series_id': series_id,
                    'instance_number': instance_number,
                    'condition': condition,  # Populate the condition based on the series description
                    'level': '',             # No level for test set, or populate based on logic
                    'x': center_x,
                    'y': center_y
                })

# Convert the list to a DataFrame
test_label_coordinates_df = pd.DataFrame(test_data_rows)

# Save the DataFrame to a CSV file
output_csv_path = '/content/test_label_coordinates_with_metadata.csv'
test_label_coordinates_df.to_csv(output_csv_path, index=False)

# Display the first few rows of the CSV to confirm
print(test_label_coordinates_df.head(100))


MARK THE X AND Y COORDINATES

In [ ]:
import os
import pydicom
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Function to find the center of a region in the DICOM image
def find_center_of_region(image):
    # Ensure the image is in uint8 format (required by Canny edge detection)
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Convert to grayscale if the image has more than one channel
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # Already grayscale

    # Use edge detection to find the region's boundaries
    edges = cv2.Canny(gray_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the region of interest
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the center of the contour
        moments = cv2.moments(largest_contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])  # x coordinate of the center
            cy = int(moments['m01'] / moments['m00'])  # y coordinate of the center
            return cx, cy
    return None, None

# Function to mark the x, y coordinates on the image
def mark_coordinates_on_image(image, x, y, output_path):
    # Ensure the image is in uint8 format (for OpenCV)
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Draw a circle at the coordinates (x, y)
    marked_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert to color to draw in color
    cv2.circle(marked_image, (x, y), radius=5, color=(0, 0, 255), thickness=-1)  # Red dot

    # Save the marked image
    cv2.imwrite(output_path, marked_image)

    # Optionally, display the image with the marked coordinates
    plt.imshow(marked_image, cmap='gray')
    plt.title(f"Marked Coordinates: ({x}, {y})")
    plt.show()

# Path to your test DICOM images (update with your actual path)
test_images_path = '/content/test_images/'

# Iterate over each test image and calculate coordinates
for root, dirs, files in os.walk(test_images_path):
    for file in files:
        if file.endswith('.dcm'):
            dicom_file = os.path.join(root, file)

            # Read the DICOM file using pydicom
            dicom_data = pydicom.dcmread(dicom_file)

            # Get the pixel data from the DICOM file
            image_array = dicom_data.pixel_array

            # Find the center of the region in the DICOM image
            center_x, center_y = find_center_of_region(image_array)

            if center_x is not None and center_y is not None:
                # Mark the coordinates on the image and save the output
                output_image_path = os.path.join('/content/marked_images/', f"marked_{file.split('.')[0]}.jpg")
                mark_coordinates_on_image(image_array, center_x, center_y, output_image_path)

                print(f"Marked and saved image: {output_image_path}")
            else:
                print(f"Skipping image {dicom_file} due to missing coordinates.")


In [ ]:
import os
import pydicom
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Function to determine the plane (Sagittal, Axial, or Coronal) from Image Orientation (Patient) tag
def determine_plane(dicom_data, tolerance=0.2):  # Increased tolerance
    orientation = dicom_data.ImageOrientationPatient

    # Convert orientation values to float for precision
    row_direction = np.array([float(val) for val in orientation[:3]])  # First three values: direction of the row
    col_direction = np.array([float(val) for val in orientation[3:]])  # Last three values: direction of the column

    # Print the orientation values for debugging
    print(f"Image Orientation (Patient): {orientation}")
    print(f"Row direction: {row_direction}, Column direction: {col_direction}")

    # Define reference vectors for Axial, Sagittal, and Coronal
    axial_row = np.array([1, 0, 0])
    axial_col = np.array([0, 1, 0])

    sagittal_row = np.array([0, 1, 0])
    sagittal_col = np.array([0, 0, -1])

    coronal_row = np.array([1, 0, 0])
    coronal_col = np.array([0, 0, -1])

    # Check if it matches Axial plane (row: X-axis, column: Y-axis)
    if np.allclose(row_direction, axial_row, atol=tolerance) and np.allclose(col_direction, axial_col, atol=tolerance):
        print("Plane: Axial")
        return "Axial"

    # Check if it matches Sagittal plane (row: Y-axis, column: Z-axis)
    elif np.allclose(row_direction, sagittal_row, atol=tolerance) and np.allclose(col_direction, sagittal_col, atol=tolerance):
        print("Plane: Sagittal")
        return "Sagittal"

    # Check if it matches Coronal plane (row: X-axis, column: Z-axis)
    elif np.allclose(row_direction, coronal_row, atol=tolerance) and np.allclose(col_direction, coronal_col, atol=tolerance):
        print("Plane: Coronal")
        return "Coronal"

    # Return Unknown if it doesn't match any plane
    else:
        print("Plane: Unknown")
        return "Unknown"

# Function to find the center of a region in the DICOM image, adjusting based on plane
def find_center_of_region(image, plane):
    # Ensure the image is in uint8 format (required by Canny edge detection)
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Convert to grayscale if the image has more than one channel
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image  # Already grayscale

    # Use edge detection to find the region's boundaries
    edges = cv2.Canny(gray_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the region of interest
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)

        # Calculate the center of the contour
        moments = cv2.moments(largest_contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])  # x coordinate of the center
            cy = int(moments['m01'] / moments['m00'])  # y coordinate of the center

            # Adjust x, y based on plane
            if plane == "Sagittal":
                print(f"Original Coordinates: x={cx}, y={cy}")
                # Swap y and x for sagittal planes to maintain consistency
                cx, cy = cy, cx
                print(f"Adjusted Coordinates (Sagittal): x={cx}, y={cy}")
            elif plane == "Coronal":
                # In coronal plane, x (left-right) is fine, y (superior-inferior) stays unchanged
                print(f"Coordinates (Coronal): x={cx}, y={cy}")
            else:
                # Axial is the default; no need to adjust
                print(f"Coordinates (Axial): x={cx}, y={cy}")
            return cx, cy
    return None, None

# Function to mark the x, y coordinates on the image
def mark_coordinates_on_image(image, x, y, plane, output_path):
    # Ensure the image is in uint8 format (for OpenCV)
    if image.dtype != np.uint8:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Draw a circle at the coordinates (x, y)
    marked_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert to color to draw in color
    cv2.circle(marked_image, (x, y), radius=5, color=(0, 0, 255), thickness=-1)  # Red dot

    # Save the marked image
    cv2.imwrite(output_path, marked_image)

    # Optionally, display the image with the marked coordinates
    plt.imshow(marked_image, cmap='gray')
    plt.title(f"Plane: {plane}, Marked Coordinates: ({x}, {y})")
    plt.show()

# Path to your test DICOM images (update with your actual path)
test_images_path = '/content/test_images/'
output_images_path = '/content/marked_images/'

# Ensure the output directory exists
os.makedirs(output_images_path, exist_ok=True)

# Iterate over each test image and calculate coordinates
for root, dirs, files in os.walk(test_images_path):
    for file in files:
        if file.endswith('.dcm'):
            dicom_file = os.path.join(root, file)

            # Read the DICOM file using pydicom
            dicom_data = pydicom.dcmread(dicom_file)

            # Get the pixel data from the DICOM file
            image_array = dicom_data.pixel_array

            # Determine the plane from the DICOM metadata
            plane = determine_plane(dicom_data)

            # Find the center of the region in the DICOM image
            center_x, center_y = find_center_of_region(image_array, plane)

            if center_x is not None and center_y is not None:
                # Mark the coordinates on the image and save the output
                output_image_path = os.path.join(output_images_path, f"marked_{file.split('.')[0]}.jpg")
                mark_coordinates_on_image(image_array, center_x, center_y, plane, output_image_path)

                print(f"Plane: {plane}, Marked and saved image: {output_image_path}")
            else:
                print(f"Skipping image {dicom_file} due to missing coordinates.")


STORE IN CSV

In [ ]:
import os
import pydicom
import numpy as np
import pandas as pd

# Function to determine the plane (Sagittal, Axial, or Coronal) from Image Orientation (Patient) tag
def determine_plane(dicom_data, tolerance=0.2):  # Increased tolerance
    orientation = dicom_data.ImageOrientationPatient

    # Convert orientation values to float for precision
    row_direction = np.array([float(val) for val in orientation[:3]])  # First three values: direction of the row
    col_direction = np.array([float(val) for val in orientation[3:]])  # Last three values: direction of the column

    # Print the orientation values for debugging
    print(f"Image Orientation (Patient): {orientation}")
    print(f"Row direction: {row_direction}, Column direction: {col_direction}")

    # Define reference vectors for Axial, Sagittal, and Coronal
    axial_row = np.array([1, 0, 0])
    axial_col = np.array([0, 1, 0])

    sagittal_row = np.array([0, 1, 0])
    sagittal_col = np.array([0, 0, -1])

    coronal_row = np.array([1, 0, 0])
    coronal_col = np.array([0, 0, -1])

    # Check if it matches Axial plane (row: X-axis, column: Y-axis)
    if np.allclose(row_direction, axial_row, atol=tolerance) and np.allclose(col_direction, axial_col, atol=tolerance):
        print("Plane: Axial")
        return "Axial"

    # Check if it matches Sagittal plane (row: Y-axis, column: Z-axis)
    elif np.allclose(row_direction, sagittal_row, atol=tolerance) and np.allclose(col_direction, sagittal_col, atol=tolerance):
        print("Plane: Sagittal")
        return "Sagittal"

    # Check if it matches Coronal plane (row: X-axis, column: Z-axis)
    elif np.allclose(row_direction, coronal_row, atol=tolerance) and np.allclose(col_direction, coronal_col, atol=tolerance):
        print("Plane: Coronal")
        return "Coronal"

    # Return Unknown if it doesn't match any plane
    else:
        print("Plane: Unknown")
        return "Unknown"

# Path to your test DICOM images (update with your actual path)
test_images_path = '/content/test_images/'

# List to store data for the CSV
csv_data = []

# Iterate over each test image and extract details
for root, dirs, files in os.walk(test_images_path):
    for file in files:
        if file.endswith('.dcm'):
            dicom_file = os.path.join(root, file)

            # Read the DICOM file using pydicom
            dicom_data = pydicom.dcmread(dicom_file)

            # Extract metadata
            study_id = dicom_data.PatientID
            series_id = dicom_data.SeriesInstanceUID
            instance_number = dicom_data.InstanceNumber

            # Determine the plane from the DICOM metadata
            plane = determine_plane(dicom_data)

            # Add the metadata and plane information to the list
            csv_data.append({
                'Study ID': study_id,
                'Series ID': series_id,
                'Instance Number': instance_number,
                'Plane': plane
            })

# Convert the list to a DataFrame
df = pd.DataFrame(csv_data)

# Path to save the CSV file
output_csv_path = '/content/dicom_planes.csv'

# Save the DataFrame to a CSV file
df.to_csv(output_csv_path, index=False)

# Print the first few rows of the CSV to confirm
print(df.head())


XAI

In [ ]:
import os
import pydicom
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# Function to find the center of a region in the DICOM image
def find_center_of_region(image):
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    edges = cv2.Canny(gray_image, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        moments = cv2.moments(largest_contour)
        if moments['m00'] != 0:
            cx = int(moments['m10'] / moments['m00'])
            cy = int(moments['m01'] / moments['m00'])
            return cx, cy
    return None, None

# Path to your test DICOM images
test_images_path = '/content/test_images/'

# List to store the test image data and coordinates
test_image_data = []
test_coords_data = []

# Iterate over each test image
for root, dirs, files in os.walk(test_images_path):
    for file in files:
        if file.endswith('.dcm'):
            dicom_file = os.path.join(root, file)
            dicom_data = pydicom.dcmread(dicom_file)
            image_array = dicom_data.pixel_array
            image_array = cv2.normalize(image_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            resized_image = cv2.resize(image_array, (64, 64))

            center_x, center_y = find_center_of_region(image_array)
            if center_x is not None and center_y is not None:
                test_image_data.append(resized_image)
                test_coords_data.append([center_x, center_y])
            else:
                print(f"Skipping {dicom_file} due to missing coordinates.")

X_test_images_resized = np.array(test_image_data).reshape(-1, 64, 64, 1)
test_coords = np.array(test_coords_data)

if X_test_images_resized.size == 0 or test_coords.size == 0:
    print("No valid test data available for prediction.")
else:
    y_pred = model.predict([X_test_images_resized, test_coords])

# Grad-CAM implementation for models with multiple inputs
def grad_cam(input_model, image, coords, layer_name):
    """
    Function to compute Grad-CAM for models with two inputs
    :param input_model: Trained model
    :param image: Image input to explain
    :param coords: Coordinates input to the model
    :param layer_name: Last convolutional layer name
    :return: Heatmap highlighting important areas
    """
    # Get the layer you want to visualize (conv2d in this case)
    grad_model = Model([input_model.inputs], [input_model.get_layer(layer_name).output, input_model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model([image, coords])  # Pass both image and coordinates
        loss = predictions[:, np.argmax(predictions[0])]

    grads = tape.gradient(loss, conv_outputs)
    guided_grads = grads[0]
    conv_outputs = conv_outputs[0]

    # Compute the weighted output for each feature map
    weights = np.mean(guided_grads, axis=(0, 1))
    cam = np.zeros(conv_outputs.shape[0:2], dtype=np.float32)

    for i, w in enumerate(weights):
        cam += w * conv_outputs[:, :, i]

    cam = np.maximum(cam, 0)
    cam = cam / cam.max()  # Normalize
    cam = cv2.resize(cam, (image.shape[1], image.shape[2]))

    return cam

# Apply Grad-CAM to the first test image
for i in range(1):  # Apply Grad-CAM to the first image as an example
    test_image = X_test_images_resized[i].reshape(1, 64, 64, 1)
    test_coords = test_coords[i].reshape(1, 2)  # Assuming coordinates are 2D

    # Apply Grad-CAM
    cam = grad_cam(model, test_image, test_coords, 'conv2d')  # Pass both image and coordinates

    # Overlay Grad-CAM on the original image
    overlayed_image = overlay_gradcam(X_test_images_resized[i], cam)

    # Display the results
    plt.imshow(overlayed_image)
    plt.title('Grad-CAM Heatmap')
    plt.show()


In [ ]:
import pandas as pd

# Load the CSV file
test_label_coordinates_df = pd.read_csv('/content/test_label_coordinates_with_metadata.csv')

# Extract the part of the series_id after the '.' and preserve it as a string
# For example, from '44036939.3481971518', we will extract '3481971518'
test_label_coordinates_df['series_number'] = test_label_coordinates_df['series_id'].apply(lambda x: str(x).split('.'))

# Save the updated DataFrame to a new CSV file without rounding off the series number
output_csv_path = '/content/test_label_coordinates_with_series_number.csv'
test_label_coordinates_df.to_csv(output_csv_path, index=False)

print(f"Updated CSV saved to {output_csv_path}")



In [ ]:
import pandas as pd
import os
import pydicom
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the test image coordinates and metadata from the CSV file
test_label_coordinates_df = pd.read_csv('/content/test_label_coordinates_with_metadata.csv')

# Function to plot the original image, coordinates, and Grad-CAM heatmap
def plot_image_with_gradcam(original_image, cam, x_coord, y_coord, alpha=0.4):
    """
    :param original_image: The original DICOM image (before resizing)
    :param cam: The Grad-CAM heatmap (from resized image)
    :param x_coord: The x coordinate from the test image
    :param y_coord: The y coordinate from the test image
    :param alpha: Transparency value for overlaying the heatmap on the original image
    """
    # Convert to color image (if grayscale) for better visualization
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)

    # Resize Grad-CAM heatmap to match the original image size
    cam_resized = cv2.resize(cam, (original_image.shape[1], original_image.shape[0]))

    # Overlay Grad-CAM heatmap on the original image
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    overlayed_image = heatmap * alpha + np.float32(original_image_rgb)
    overlayed_image = overlayed_image / np.max(overlayed_image)

    # Plot original image with coordinates, heatmap, and combined view
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # Plot original image with x, y coordinates marked
    axs[0].imshow(original_image, cmap='gray')
    axs[0].scatter(x_coord, y_coord, color='red', marker='x')  # Mark x, y coordinate
    axs[0].set_title(f"Original Image\nCoordinates: ({x_coord}, {y_coord})")

    # Plot Grad-CAM heatmap
    axs[1].imshow(cam_resized, cmap='jet')
    axs[1].set_title("Grad-CAM Heatmap")

    # Plot image with Grad-CAM heatmap overlay
    axs[2].imshow(overlayed_image)
    axs[2].scatter(x_coord, y_coord, color='red', marker='x')  # Mark x, y coordinate
    axs[2].set_title(f"Image with Grad-CAM Overlay\nCoordinates: ({x_coord}, {y_coord})")

    plt.show()

# Function to find the DICOM file based on study_id and series_number
def find_dicom_file(study_id, series_number, base_dir):
    """
    Finds the DICOM file based on the study ID and series number.
    :param study_id: The study ID of the image.
    :param series_number: The series number of the image.
    :param base_dir: Base directory where DICOM files are stored.
    :return: Path to the DICOM file if found, else None.
    """
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.dcm') and study_id in root and series_id in root:
                return os.path.join(root, file)
    return None

# Process just 5 images from the CSV file
for idx, row in test_label_coordinates_df.head(100).iterrows():
    study_id = str(row['study_id'])
    series_id = str(row['series_id'])

    # Find the DICOM file based on study_id and series_number
    dicom_file_path = find_dicom_file(study_id, series_id, '/content/test_images/')

    if dicom_file_path:
        # Read the DICOM file
        dicom_data = pydicom.dcmread(dicom_file_path)
        original_image_array = dicom_data.pixel_array

        # Resize the image to (64, 64) only for the model input
        resized_image = cv2.resize(original_image_array, (64, 64))

        # Extract x, y coordinates from the CSV
        x_coord = row['x']
        y_coord = row['y']

        # Assuming the `model` and `grad_cam` function are already defined for Grad-CAM
        # Apply Grad-CAM on the resized image
        test_image_input = resized_image.reshape(1, resized_image.shape[0], resized_image.shape[1], 1)
        test_coords_input = np.array([[x_coord, y_coord]])
        cam = grad_cam(model, test_image_input, test_coords_input, 'conv2d')

        # Plot the original image, x/y coordinates, and Grad-CAM heatmap
        plot_image_with_gradcam(original_image_array, cam, x_coord, y_coord)
    else:
        print(f"DICOM file not found for study ID {study_id} and series id {series_id}")


TRYING TO FIND THE CONDITION ADN THE LEVEL OF SEVERITY

In [ ]:
# import os
# import pandas as pd
# import numpy as np
# import pydicom
# import cv2
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
# from tensorflow.keras.applications import ResNet50
# from sklearn.model_selection import train_test_split

# # Load the CSV files
# train_label_coordinates_df = pd.read_csv('/content/train_label_coordinates.csv')
# train_data_df = pd.read_csv('/content/train.csv')
# train_series_descriptions_df = pd.read_csv('/content/train_series_descriptions.csv')

# # Function to preprocess DICOM images
# def preprocess_image(dicom_file, target_size=(224, 224)):
#     dicom_data = pydicom.dcmread(dicom_file)
#     image = dicom_data.pixel_array
#     image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
#     image_resized = cv2.resize(image, target_size)
#     image_resized = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)
#     return image_resized

# # Preprocess the data and labels
# X = []
# y_stenosis = []
# y_neural_left = []
# y_neural_right = []
# y_subarticular_left = []
# y_subarticular_right = []

# for _, row in train_label_coordinates_df.iterrows():
#     # Match series ID to load image and series description
#     series_id = row['series_id']
#     series_desc_row = train_series_descriptions_df[train_series_descriptions_df['series_id'] == series_id]

#     # Fetch corresponding labels from the train data
#     condition_row = train_data_df[train_data_df['study_id'] == row['study_id']]

#     if not series_desc_row.empty and not condition_row.empty:
#         dicom_file = os.path.join('/content/train_images', str(row['study_id']), str(series_id), f'{row["instance_number"]}.dcm')

#         if os.path.exists(dicom_file):
#             # Preprocess and append image
#             X.append(preprocess_image(dicom_file))

#             # Correct the level format (replace '/' with '_')
#             level = row["level"].replace('/', '_').lower()

#             # Append labels
#             y_stenosis.append(condition_row[f'spinal_canal_stenosis_{level}'].values[0])
#             y_neural_left.append(condition_row[f'left_neural_foraminal_narrowing_{level}'].values[0])
#             y_neural_right.append(condition_row[f'right_neural_foraminal_narrowing_{level}'].values[0])
#             y_subarticular_left.append(condition_row[f'left_subarticular_stenosis_{level}'].values[0])
#             y_subarticular_right.append(condition_row[f'right_subarticular_stenosis_{level}'].values[0])



# # Convert to numpy arrays
# X = np.array(X)
# X = X / 255.0  # Normalize images

# # One-hot encode labels (assuming severity levels like Normal/Mild, Moderate, Severe)
# label_mapping = {'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2}

# y_stenosis = np.array([label_mapping[severity] for severity in y_stenosis])
# y_neural_left = np.array([label_mapping[severity] for severity in y_neural_left])
# y_neural_right = np.array([label_mapping[severity] for severity in y_neural_right])
# y_subarticular_left = np.array([label_mapping[severity] for severity in y_subarticular_left])
# y_subarticular_right = np.array([label_mapping[severity] for severity in y_subarticular_right])

# # One-hot encode the labels
# y_stenosis_encoded = to_categorical(y_stenosis, num_classes=3)
# y_neural_left_encoded = to_categorical(y_neural_left, num_classes=3)
# y_neural_right_encoded = to_categorical(y_neural_right, num_classes=3)
# y_subarticular_left_encoded = to_categorical(y_subarticular_left, num_classes=3)
# y_subarticular_right_encoded = to_categorical(y_subarticular_right, num_classes=3)

# # Split into training and validation sets
# X_train, X_val, y_stenosis_train, y_stenosis_val, y_neural_left_train, y_neural_left_val, y_neural_right_train, y_neural_right_val, y_subarticular_left_train, y_subarticular_left_val, y_subarticular_right_train, y_subarticular_right_val = train_test_split(
#     X, y_stenosis_encoded, y_neural_left_encoded, y_neural_right_encoded, y_subarticular_left_encoded, y_subarticular_right_encoded, test_size=0.2, random_state=42
# )

# # Define the model
# input_shape = (224, 224, 3)
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
# x = Flatten()(base_model.output)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.5)(x)

# # Output layers for each condition
# output_stenosis = Dense(3, activation='softmax', name='stenosis_output')(x)
# output_neural_left = Dense(3, activation='softmax', name='neural_left_output')(x)
# output_neural_right = Dense(3, activation='softmax', name='neural_right_output')(x)
# output_subarticular_left = Dense(3, activation='softmax', name='subarticular_left_output')(x)
# output_subarticular_right = Dense(3, activation='softmax', name='subarticular_right_output')(x)

# # Multi-output model
# model = Model(inputs=base_model.input, outputs=[output_stenosis, output_neural_left, output_neural_right, output_subarticular_left, output_subarticular_right])

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(
#     X_train,
#     [y_stenosis_train, y_neural_left_train, y_neural_right_train, y_subarticular_left_train, y_subarticular_right_train],
#     epochs=10,
#     batch_size=32,
#     validation_data=(X_val, [y_stenosis_val, y_neural_left_val, y_neural_right_val, y_subarticular_left_val, y_subarticular_right_val])
# )


In [ ]:
import os
import pandas as pd
import numpy as np
import pydicom
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import ResNet50

# Function to preprocess DICOM images
def preprocess_image(dicom_file, target_size=(224, 224)):
    dicom_data = pydicom.dcmread(dicom_file)
    image = dicom_data.pixel_array
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    image_resized = cv2.resize(image, target_size)
    image_resized = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)
    return image_resized

# Load the CSV files
train_label_coordinates_df = pd.read_csv('/content/train_label_coordinates.csv')
train_data_df = pd.read_csv('/content/train.csv')
train_series_descriptions_df = pd.read_csv('/content/train_series_descriptions.csv')

# Preprocess the data and labels
X = []
y_stenosis = []
y_neural_left = []
y_neural_right = []
y_subarticular_left = []
y_subarticular_right = []

for _, row in train_label_coordinates_df.iterrows():
    # Match series ID to load image and series description
    series_id = row['series_id']
    series_desc_row = train_series_descriptions_df[train_series_descriptions_df['series_id'] == series_id]

    # Fetch corresponding labels from the train data
    condition_row = train_data_df[train_data_df['study_id'] == row['study_id']]

    if not series_desc_row.empty and not condition_row.empty:
        dicom_file = os.path.join('/content/train_images', str(row['study_id']), str(series_id), f'{row["instance_number"]}.dcm')

        if os.path.exists(dicom_file):
            # Preprocess and append image
            X.append(preprocess_image(dicom_file))

            # Correct the level format (replace '/' with '_')
            level = row["level"].replace('/', '_').lower()

            # Append labels
            y_stenosis.append(condition_row[f'spinal_canal_stenosis_{level}'].values[0])
            y_neural_left.append(condition_row[f'left_neural_foraminal_narrowing_{level}'].values[0])
            y_neural_right.append(condition_row[f'right_neural_foraminal_narrowing_{level}'].values[0])
            y_subarticular_left.append(condition_row[f'left_subarticular_stenosis_{level}'].values[0])
            y_subarticular_right.append(condition_row[f'right_subarticular_stenosis_{level}'].values[0])

# Convert to numpy arrays
X = np.array(X)
X = X / 255.0  # Normalize images

# One-hot encode labels (assuming severity levels like Normal/Mild, Moderate, Severe)
label_mapping = {'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2}

y_stenosis = np.array([label_mapping[severity] for severity in y_stenosis])
y_neural_left = np.array([label_mapping[severity] for severity in y_neural_left])
y_neural_right = np.array([label_mapping[severity] for severity in y_neural_right])
y_subarticular_left = np.array([label_mapping[severity] for severity in y_subarticular_left])
y_subarticular_right = np.array([label_mapping[severity] for severity in y_subarticular_right])

# One-hot encode the labels
y_stenosis_encoded = to_categorical(y_stenosis, num_classes=3)
y_neural_left_encoded = to_categorical(y_neural_left, num_classes=3)
y_neural_right_encoded = to_categorical(y_neural_right, num_classes=3)
y_subarticular_left_encoded = to_categorical(y_subarticular_left, num_classes=3)
y_subarticular_right_encoded = to_categorical(y_subarticular_right, num_classes=3)


In [ ]:
# Split into training and validation sets
X_train, X_val, y_stenosis_train, y_stenosis_val, y_neural_left_train, y_neural_left_val, y_neural_right_train, y_neural_right_val, y_subarticular_left_train, y_subarticular_left_val, y_subarticular_right_train, y_subarticular_right_val = train_test_split(
    X, y_stenosis_encoded, y_neural_left_encoded, y_neural_right_encoded, y_subarticular_left_encoded, y_subarticular_right_encoded, test_size=0.2, random_state=42
)




In [ ]:
# Define the model
input_shape = (224, 224, 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layers for each condition
output_stenosis = Dense(3, activation='softmax', name='stenosis_output')(x)
output_neural_left = Dense(3, activation='softmax', name='neural_left_output')(x)
output_neural_right = Dense(3, activation='softmax', name='neural_right_output')(x)
output_subarticular_left = Dense(3, activation='softmax', name='subarticular_left_output')(x)
output_subarticular_right = Dense(3, activation='softmax', name='subarticular_right_output')(x)

# Multi-output model
model = Model(inputs=base_model.input, outputs=[output_stenosis, output_neural_left, output_neural_right, output_subarticular_left, output_subarticular_right])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    X_train,
    [y_stenosis_train, y_neural_left_train, y_neural_right_train, y_subarticular_left_train, y_subarticular_right_train],
    epochs=10,
    batch_size=32,
    validation_data=(X_val, [y_stenosis_val, y_neural_left_val, y_neural_right_val, y_subarticular_left_val, y_subarticular_right_val])
)

### Making Predictions for Test Data and Preparing Submission ###

# Load the test data from the test series descriptions
test_series_descriptions_df = pd.read_csv('/content/test_series_descriptions.csv')

# Preprocess test images
X_test = []

for _, row in test_series_descriptions_df.iterrows():
    dicom_file = os.path.join('/content/test_images', str(row['study_id']), str(row['series_id']), f'{row["instance_number"]}.dcm')
    if os.path.exists(dicom_file):
        X_test.append(preprocess_image(dicom_file))

X_test = np.array(X_test) / 255.0  # Normalize images

# Make predictions on the test set
y_pred_stenosis, y_pred_neural_left, y_pred_neural_right, y_pred_subarticular_left, y_pred_subarticular_right = model.predict(X_test)

# Convert predictions to class indices (0, 1, 2) for each condition
y_pred_stenosis_classes = np.argmax(y_pred_stenosis, axis=1)
y_pred_neural_left_classes = np.argmax(y_pred_neural_left, axis=1)
y_pred_neural_right_classes = np.argmax(y_pred_neural_right, axis=1)
y_pred_subarticular_left_classes = np.argmax(y_pred_subarticular_left, axis=1)
y_pred_subarticular_right_classes = np.argmax(y_pred_subarticular_right, axis=1)

# Prepare the submission file
submission_df = pd.DataFrame({
    'study_id': test_label_coordinates_df['study_id'],
    'spinal_canal_stenosis_l1_l2': y_pred_stenosis,
    'spinal_canal_stenosis_l2_l3': y_pred_stenosis,
    'spinal_canal_stenosis_l3_l4': y_pred_stenosis,
    'spinal_canal_stenosis_l4_l5': y_pred_stenosis,
    'spinal_canal_stenosis_l5_s1': y_pred_stenosis,
    'left_neural_foraminal_narrowing_l1_l2': y_pred_neural_left,
    'left_neural_foraminal_narrowing_l2_l3': y_pred_neural_left,
    'left_neural_foraminal_narrowing_l3_l4': y_pred_neural_left,
    'left_neural_foraminal_narrowing_l4_l5': y_pred_neural_left,
    'left_neural_foraminal_narrowing_l5_s1': y_pred_neural_left,
    'right_neural_foraminal_narrowing_l1_l2': y_pred_neural_right,
    'right_neural_foraminal_narrowing_l2_l3': y_pred_neural_right,
    'right_neural_foraminal_narrowing_l3_l4': y_pred_neural_right,
    'right_neural_foraminal_narrowing_l4_l5': y_pred_neural_right,
    'right_neural_foraminal_narrowing_l5_s1': y_pred_neural_right,
    'left_subarticular_stenosis_l1_l2': y_pred_subarticular_left,
    'left_subarticular_stenosis_l2_l3': y_pred_subarticular_left,
    'left_subarticular_stenosis_l3_l4': y_pred_subarticular_left,
    'left_subarticular_stenosis_l4_l5': y_pred_subarticular_left,
    'left_subarticular_stenosis_l5_s1': y_pred_subarticular_left,
    'right_subarticular_stenosis_l1_l2': y_pred_subarticular_right,
    'right_subarticular_stenosis_l2_l3': y_pred_subarticular_right,
    'right_subarticular_stenosis_l3_l4': y_pred_subarticular_right,
    'right_subarticular_stenosis_l4_l5': y_pred_subarticular_right,
    'right_subarticular_stenosis_l5_s1': y_pred_subarticular_right
})

# Save the submission to a CSV file
submission_file_path = '/content/submission.csv'
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file saved to {submission_file_path}")